<a href="https://colab.research.google.com/github/DasariRishikesh/Compression_NeuralNets/blob/main/MP_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**MINI PROJECT-Compression Neural Nets**
*Building Neural Network*

In [ ]:
pip install -q tensorflow-model-optimization

In [ ]:
import tempfile
import os

import tensorflow as tf
import numpy as np

from tensorflow import keras

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=4,
  validation_split=0.1,
)

Epoch 1/4
1688/1688 [==============================] - 27s 16ms/step - loss: 0.2934 - accuracy: 0.9168 - val_loss: 0.1233 - val_accuracy: 0.9675
Epoch 2/4
1688/1688 [==============================] - 23s 13ms/step - loss: 0.1161 - accuracy: 0.9668 - val_loss: 0.0888 - val_accuracy: 0.9757
Epoch 3/4
1688/1688 [==============================] - 24s 14ms/step - loss: 0.0830 - accuracy: 0.9764 - val_loss: 0.0702 - val_accuracy: 0.9823
Epoch 4/4
1688/1688 [==============================] - 25s 15ms/step - loss: 0.0681 - accuracy: 0.9796 - val_loss: 0.0643 - val_accuracy: 0.9828


In [ ]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Baseline test accuracy: 0.9775000214576721
Saved baseline model to: /tmp/tmpeni0_14r.h5


#Pruning the trained Models

In [ ]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

batch_size = 128
epochs = 2
validation_split = 0.1

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

#setting a pruing model
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()          #recompiling them

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_reshape  (None, 28, 28, 1)        1         
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_conv2d   (None, 26, 26, 12)       230       
 (PruneLowMagnitude)                                             
                                                                 
 prune_low_magnitude_max_poo  (None, 13, 13, 12)       1         
 ling2d (PruneLowMagnitude)                                      
                                                                 
 prune_low_magnitude_flatten  (None, 2028)             1         
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_dense (  (None, 10)               4

In [ ]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/2
422/422 [==============================] - 18s 35ms/step - loss: 0.1048 - accuracy: 0.9707 - val_loss: 0.1215 - val_accuracy: 0.9690
Epoch 2/2
422/422 [==============================] - 16s 37ms/step - loss: 0.1189 - accuracy: 0.9692 - val_loss: 0.0982 - val_accuracy: 0.9742


In [ ]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.9775000214576721
Pruned test accuracy: 0.9728000164031982


Before pruning the model accuracy (i.e baseline accuracy) = 0.9775
After introducing the pruning model(i.e model_ pruning) and applying them over the dataset till 2 epochs with sparty of 10% i.e 10% of the weights in hidden layers becomes '0', and produces the accuracy range = 0.9728